# HW3 IMDB + Two Layer Net

- 수업시간에 다룬 Two Layer Net를 이용하여 HW2에서 다룬 IMDB 데이터를 학습하고 테스트 하는 프로그램 작성
- IMDB 데이터에 필요한 vocab.txt는 제공됨
- IMDB 데이터는 이 노트북과 같은 디렉토리에 있고 현재 디렉토리 밑에 txt_sentoken/pos, txt_sentoken/neg에 해당 파일이 있는 것으로 하고 프로그래밍할 것
- 제출시에는 데이터 파일은 제출할 필요가 없음
- Two Layer Net에서는 필요한 클래스, 모듈은 이 노트북에 명시되어야 함. import해서는 안됨
- 70%를 train

In [1]:
## IMDB 데이터를 위한 함수 구현 
## https://machinelearningmastery.com/prepare-movie-review-data-sentiment-analysis/ 에서 그대로 가져다 사용할 수 있음
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
import itertools
import numpy as np

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

# save list to file
def save_list(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
    # load the doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [w for w in tokens if w in vocab]
    return ' '.join(tokens)

# load all docs in a directory
def process_docs(directory, vocab):
    lines = list()
    # walk through all files in the folder
    for filename in listdir(directory):
        # skip files that do not have the right extension
        if not filename.endswith(".txt"):
            continue
        # create the full path of the file to open
        path = directory + '/' + filename
        # load and clean the doc
        line = doc_to_line(path, vocab)
        # add to list
        lines.append(line)
    return lines


In [2]:
#encoding
def texts_to_sequence(train_docs):
    #flatten train_docs for encoding
    flat_words = list(itertools.chain(*train_docs))
    flat_words = sorted(list(set(flat_words)))
    #encode char->int, decode int->char
    char_to_int = dict((c, i) for i, c in enumerate(flat_words)) #아마 char, int로 input 들어옴
    seq_out = list()
    for line in train_docs:
        encoded_seq = [char_to_int[char]/len(flat_words) for char in line]
        seq_out.append(encoded_seq)

    return seq_out

#padding
def pad_sequences(sequences, number, width):  #width는 max길이 한번 찾아보자
    padd_seq_out = list()
    #max_length = max([len(s.split()) for s in sequences])
    for line in sequences:
        #print (len(line))
        line.extend(np.repeat(number, width - len(line)))     #이게 0으로 채우는건가...? num=0주면 되나
        #line = [line.extend([number] * (width - len(line)))]
        padd_seq_out.append(line)
    #sequences.extend([number] * (width - len(sequences)))
    return padd_seq_out

def one_hot_encoding(x):

    output = np.zeros([np.size(x),2])    
    for i,index in enumerate(x):
        output[i,index]=1
    return output

In [3]:
## vocab.txt를 불러와서 실제로 사용될 vocab을 load
# load vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
# prepare negative reviews
negative_lines = process_docs('txt_sentoken/neg', vocab)
# prepare positive reviews
positive_lines = process_docs('txt_sentoken/pos', vocab)
train_lines=negative_lines+positive_lines

In [4]:
encoded=texts_to_sequence(train_lines)

In [5]:
maxlen = max([len(s) for s in encoded])
#print(maxlen)

NameError: name 'encode' is not defined

In [ ]:
#pad all sentences
all_x=pad_sequences(encoded, 0, maxlen)

In [ ]:
##load all training reviews
train_x=all_x[0:700]+all_x[1000:1700]
train_x=np.array(train_x)

In [ ]:
##make train label
trainlabel=list()
for i in range(700): trainlabel.append(0)  #neg=0->[1,0]
for i in range(700): trainlabel.append(1) #pos=1->[0,1]
train_t=one_hot_encoding(trainlabel)

In [ ]:
## load all test reviews
test_x=all_x[700:1000]+all_x[1700:2000]
test_x=np.array(test_x)

In [ ]:
##make test label
testlabel=list()
for i in range(300): testlabel.append(0)  #neg=0->[1,0]
for i in range(300): testlabel.append(1) #pos=1->[0,1]
test_t=one_hot_encoding(testlabel)

In [ ]:
##deep_learning_from_scratch github에 ch05에 있는 two_layer_net.py를 근간으로 하고 
## 필요한 class는 common 디렉토리 밑에 있는 functions.py, layer.py 등을 참조
from collections import OrderedDict

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = OrderedDict()
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = {}
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
        

    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx    
    

class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx    
    

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx    
    
    
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx
    
    

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
    
    

def sigmoid(x):
    return 1 / (1 + np.exp(-x))    
    
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

def _numerical_gradient_1d(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        
    return grad


def numerical_gradient_2d(f, X):
    if X.ndim == 1:
        return _numerical_gradient_1d(f, X)
    else:
        grad = np.zeros_like(X)
        
        for idx, x in enumerate(X):
            grad[idx] = _numerical_gradient_1d(f, x)
        
        return grad
    

def numerical_gradient(f, x):
    loss_W = lambda W: self.loss(x, t)
        
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
    return grads

In [ ]:
# training 
## ch05에 있는 train_neuralnet.py를 이 데이터에 맞도록 수정하여 사용.
#training accuracy와 test accuracy를 매 epoch마다 출력


# coding: utf-8
import sys, os
sys.path.append(os.pardir)
import numpy as np

# 데이터 읽기
network = TwoLayerNet(input_size=maxlen, hidden_size=50, output_size=2)

iters_num = 10000
train_size = train_x.shape[0]
batch_size = 100
learning_rate = 0.01

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = train_x[batch_mask] 
    t_batch = train_t[batch_mask] 
    
    # 기울기 계산
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(train_x, train_t)
        test_acc = network.accuracy(test_x, test_t)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)